<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../../README.md)

7.1 : Find Influencers on Twitter
================================

### Overview
Find 'influencers' on Twitter graph

### Depends On
None

### Run time
20 mins

### Lab Setup

This lab uses the GraphFrames spark package.  It is currently NOT part of the default spark framework.

We use this for two reasons:

1. GraphFrames likely will be the basis of future graph processing in spark.

2. RDD-based graphx has no python API, whereas GraphFrames does have a python API. As this jupyter notebook is python that makes it required.

So, to run this we have two choices

**option 1 : Jupyter**

**Note:** 
Jupyter lab will be already running on the port 8888.
So, kill the process first.
```bash
$ sudo netstat -plnt | grep 8888
The process id will be shown in the output.Replace process id in the kill command

$ sudo kill -9 process id
```

```bash
$  PYSPARK_PYTHON=python3 PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/spark/bin/pyspark --packages graphframes:graphframes:0.7.0-spark2.4-s_2.11
```

**option 2 : PySpark (command line)**
```bash
$  PYSPARK_PYTHON=python3  ~/spark/bin/pyspark --packages graphframes:graphframes:0.7.0-spark2.4-s_2.11
```

Replace the version on graphframes with your latest.

With that said, let's start:

## Step 1: Build the following twitter graph
Here is some real world data:
<img src="../assets/images/7.1a.png" style="border: 5px solid grey; max-width:100%;"/>

We are using data from a real Twitter account, if you want, you can use yours

#### Import the necessary libraries

In [3]:
# %AddDeps graphframes graphframes 0.7.0-spark2.4-s_2.11
#%lsmagic

In [4]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
sc = spark.sparkContext

Initializing Spark...
Spark found in :  /home/ubuntu/spark
Spark config:
	 executor.memory=2g
	some_property=some_value
	spark.app.name=TestApp
	spark.files=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.jars=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.master=local[*]
	spark.repl.local.jars=file:///home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,file:///home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.sql.warehouse.dir=/tmp/tmprusly6b4
	spark.submit.deployMode=client
	spark.submit.pyFiles=/home/ubuntu/.ivy2/jars/graphframes_graphframes-0.7.0-spark2.4-s_2.11.jar,/home/ubuntu/.ivy2/jars/org.slf4j_slf4j-api-1.7.16.jar
	spark.ui.showConsoleProgress=true
Spark UI running on port 4047


In [5]:
from graphframes import *

#### Construct the array of vertices

 Data structure: twitter handle, number of followers, gender of the tweeter

In [7]:
vertices = spark.createDataFrame([
        (1, "@markkerzner", 309, "M"),  # (Id, Name, no of followers, gender)
        (2, "@mjbrender", 3101, "M"),
        (3, "@dridisahar1", 27, "F"),
        (4, "@dez_blanchfield ", 38600, "M"),
        (5, "@ch_doig ", 519, "F"),
        (6, "@Sunitha_Packt ", 332, "F"),
        (7, "@WibiData ", 2477, "N")  # Name here is company, so gender is neutral
], ["id", "Name", "followers", "gender"]) 

vertices.show()

+---+-----------------+---------+------+
| id|             Name|followers|gender|
+---+-----------------+---------+------+
|  1|     @markkerzner|      309|     M|
|  2|       @mjbrender|     3101|     M|
|  3|     @dridisahar1|       27|     F|
|  4|@dez_blanchfield |    38600|     M|
|  5|        @ch_doig |      519|     F|
|  6|  @Sunitha_Packt |      332|     F|
|  7|       @WibiData |     2477|     N|
+---+-----------------+---------+------+



####  Construct the array of edges

On this step, these are all my followers, so they connect to me

In [8]:
edges = spark.createDataFrame([
        (1, 2, 7), # (src, dest, retweets)
        (1, 3, 2),
        (1, 4, 4),
        (1, 5, 3),
        (1, 6, 1),
        (1, 7, 2)
], ["src", "dst", "retweets"])

edges.show()

+---+---+--------+
|src|dst|retweets|
+---+---+--------+
|  1|  2|       7|
|  1|  3|       2|
|  1|  4|       4|
|  1|  5|       3|
|  1|  6|       1|
|  1|  7|       2|
+---+---+--------+



#### Construct the graph from the vertices and edges

In [9]:
graph = GraphFrame(vertices, edges)

## Step 2 : Analyzing Graph
#### Print graph

In [10]:
# Vertices
graph.vertices.show()

+---+-----------------+---------+------+
| id|             Name|followers|gender|
+---+-----------------+---------+------+
|  1|     @markkerzner|      309|     M|
|  2|       @mjbrender|     3101|     M|
|  3|     @dridisahar1|       27|     F|
|  4|@dez_blanchfield |    38600|     M|
|  5|        @ch_doig |      519|     F|
|  6|  @Sunitha_Packt |      332|     F|
|  7|       @WibiData |     2477|     N|
+---+-----------------+---------+------+



In [11]:
# Edges
graph.edges.show()

+---+---+--------+
|src|dst|retweets|
+---+---+--------+
|  1|  2|       7|
|  1|  3|       2|
|  1|  4|       4|
|  1|  5|       3|
|  1|  6|       1|
|  1|  7|       2|
+---+---+--------+



In [12]:
# triplets
graph.triplets.show(truncate=False)

+-------------------------+---------+--------------------------------+
|src                      |edge     |dst                             |
+-------------------------+---------+--------------------------------+
|[1, @markkerzner, 309, M]|[1, 7, 2]|[7, @WibiData , 2477, N]        |
|[1, @markkerzner, 309, M]|[1, 6, 1]|[6, @Sunitha_Packt , 332, F]    |
|[1, @markkerzner, 309, M]|[1, 5, 3]|[5, @ch_doig , 519, F]          |
|[1, @markkerzner, 309, M]|[1, 3, 2]|[3, @dridisahar1, 27, F]        |
|[1, @markkerzner, 309, M]|[1, 2, 7]|[2, @mjbrender, 3101, M]        |
|[1, @markkerzner, 309, M]|[1, 4, 4]|[4, @dez_blanchfield , 38600, M]|
+-------------------------+---------+--------------------------------+



## Step 3 : Query the graph

#### Filter out male followers

In [13]:
graph.vertices.filter("gender != 'M'").show()

+---+---------------+---------+------+
| id|           Name|followers|gender|
+---+---------------+---------+------+
|  3|   @dridisahar1|       27|     F|
|  5|      @ch_doig |      519|     F|
|  6|@Sunitha_Packt |      332|     F|
|  7|     @WibiData |     2477|     N|
+---+---------------+---------+------+



#### Find my significant followers

In [14]:
graph.vertices.filter("followers > 1000").show() 

+---+-----------------+---------+------+
| id|             Name|followers|gender|
+---+-----------------+---------+------+
|  2|       @mjbrender|     3101|     M|
|  4|@dez_blanchfield |    38600|     M|
|  7|       @WibiData |     2477|     N|
+---+-----------------+---------+------+



#### Find those followers who do enough re-tweeets for me

In [15]:
graph.edges.filter("retweets > 5").show()

graph.edges.filter("retweets > 5").count()

+---+---+--------+
|src|dst|retweets|
+---+---+--------+
|  1|  2|       7|
+---+---+--------+



1

#### Count my male and female followers

In [16]:
num_male = graph.vertices.filter("gender == 'M'").count()
num_female = graph.vertices.filter("gender == 'F'").count()
print ('Males %d, Females %d' % (num_male, num_female))

Males 3, Females 3
